<a href="https://colab.research.google.com/github/prakash-tech89/Data-Science/blob/master/Python/Hackathon/Sentiment_Analysis_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data load

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [0]:
train_df=pd.read_csv("/content/review_train.csv",sep=",")
test_df=pd.read_csv("/content/review_test.csv",sep=",")

In [0]:
train_df["source"] = "train"
test_df["source"] = "test"
df = pd.concat([train_df,test_df])

# Using NLTK

## Loading Necessary Library

In [0]:
!pip install textsearch
!pip install contractions
!pip install tqdm
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
!python -m spacy validate

✔ Loaded compatibility table

====================== Installed models (spaCy v2.1.8) ======================
ℹ spaCy installation: /usr/local/lib/python3.6/dist-packages/spacy

TYPE      NAME             MODEL            VERSION                            
package   en-core-web-sm   en_core_web_sm   2.1.0   ✔
link      en               en_core_web_sm   2.1.0   ✔



In [0]:
!python -m spacy download en


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

## Pre-Processing

In [0]:
import re
from nltk.stem import WordNetLemmatizer
import contractions
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize

wnl = WordNetLemmatizer()

def tokenize_sent(text):
    return (nltk.sent_tokenize(text))

def to_lower(text):
    return text.lower()

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text
def expand_contractions(text):
    return contractions.fix(text)
def lemmatized_text(tokens):
    return wnl.lemmatize(tokens)
def tokenize_word(text):
    return nltk.word_tokenize(text)
def pos_tag(text):
    return nltk.pos_tag(text)
def pos_tag_wordnet(tagged_tokens):
    tag_map = {'j': wordnet.ADJ, 'v': wordnet.VERB, 'n': wordnet.NOUN, 'r': wordnet.ADV}
    new_tagged_tokens = [(word, tag_map.get(tag[0].lower(), wordnet.NOUN))
                            for word, tag in tagged_tokens]
    return new_tagged_tokens
def remove_stopwords(text, is_lower_case=False, stopwords=None):
    if not stopwords:
        stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
def normalize_document(doc):
    stop_words = nltk.corpus.stopwords.words('english')
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return np.array(doc)



In [0]:
df["pre-processed"]=df.apply(lambda x : (tokenize_sent(str(x["Text"]))),axis=1)
df["pre-processed"].head(7)

0    [I got a free sample of these once, and now--w...
1    [I used to get this Tea when I lived in Washin...
2    [This is my all time favorite 'grab and go' sn...
3    [This flavor is very good and unexpected., The...
4    [thrilled to have this assortment as i got the...
5    [I love the flavor of this coffee., The only d...
6    [Like the last review, I have a mixture of dog...
Name: pre-processed, dtype: object

In [0]:
df["pre-processed"]=df.apply(lambda x : remove_special_characters(str(x["pre-processed"]),remove_digits=False) ,axis=1)
df["pre-processed"].head(7)

0    I got a free sample of these once and nowwere ...
1    I used to get this Tea when I lived in Washing...
2    This is my all time favorite grab and go snack...
3    This flavor is very good and unexpected The fl...
4    thrilled to have this assortment as i got the ...
5    I love the flavor of this coffee The only draw...
6    Like the last review I have a mixture of dogs ...
Name: pre-processed, dtype: object

In [0]:
df["pre-processed"]=df.apply(lambda x : expand_contractions(x["pre-processed"]),axis=1)
df["pre-processed"].head(7)

0    I got a free sample of these once and nowwere ...
1    I used to get this Tea when I lived in Washing...
2    This is my all time favorite grab and go snack...
3    This flavor is very good and unexpected The fl...
4    thrilled to have this assortment as i got the ...
5    I love the flavor of this coffee The only draw...
6    Like the last review I have a mixture of dogs ...
Name: pre-processed, dtype: object

In [0]:

df["pre-processed"]=df.apply(lambda x : to_lower(x["pre-processed"]),axis=1)
df["pre-processed"].head(7)

0    i got a free sample of these once and nowwere ...
1    i used to get this tea when i lived in washing...
2    this is my all time favorite grab and go snack...
3    this flavor is very good and unexpected the fl...
4    thrilled to have this assortment as i got the ...
5    i love the flavor of this coffee the only draw...
6    like the last review i have a mixture of dogs ...
Name: pre-processed, dtype: object

In [0]:
df["pre-processed"]=df.apply(lambda x : remove_stopwords(str(x["pre-processed"])),axis=1)
df["pre-processed"].head(7)

0    got free sample nowwere subscribe save program...
1    used get tea lived washington state missed nie...
2    time favorite grab go snack bar leave sugar ru...
3    flavor good unexpected flavor cheese garlic go...
4    thrilled assortment got chance choose another ...
5    love flavor coffee drawback set largest cup wa...
6    like last review mixture dogs 2 chihuahuas eat...
Name: pre-processed, dtype: object

In [0]:
df["tokenized"]=df.apply(lambda x : tokenize_word(x["pre-processed"]),axis=1)
df["tokenized"].head(7)



0    [got, free, sample, nowwere, subscribe, save, ...
1    [used, get, tea, lived, washington, state, mis...
2    [time, favorite, grab, go, snack, bar, leave, ...
3    [flavor, good, unexpected, flavor, cheese, gar...
4    [thrilled, assortment, got, chance, choose, an...
5    [love, flavor, coffee, drawback, set, largest,...
6    [like, last, review, mixture, dogs, 2, chihuah...
Name: tokenized, dtype: object

In [0]:
df["pre-processed-tag"]=df.apply(lambda x : pos_tag(x["tokenized"]),axis=1)
df["pre-processed-tag"].head(7)

0    [(got, VBD), (free, JJ), (sample, NN), (nowwer...
1    [(used, VBN), (get, VB), (tea, JJ), (lived, JJ...
2    [(time, NN), (favorite, JJ), (grab, NN), (go, ...
3    [(flavor, NN), (good, JJ), (unexpected, JJ), (...
4    [(thrilled, JJ), (assortment, NN), (got, VBD),...
5    [(love, VB), (flavor, NN), (coffee, NN), (draw...
6    [(like, IN), (last, JJ), (review, NN), (mixtur...
Name: pre-processed-tag, dtype: object

In [0]:
df["pre-processed-wordnet"]=df.apply(lambda x : pos_tag_wordnet(x["pre-processed-tag"]),axis=1)
df["pre-processed-wordnet"].head(7)

0    [(got, v), (free, a), (sample, n), (nowwere, r...
1    [(used, v), (get, v), (tea, a), (lived, a), (w...
2    [(time, n), (favorite, a), (grab, n), (go, v),...
3    [(flavor, n), (good, a), (unexpected, a), (fla...
4    [(thrilled, a), (assortment, n), (got, v), (ch...
5    [(love, v), (flavor, n), (coffee, n), (drawbac...
6    [(like, n), (last, a), (review, n), (mixture, ...
Name: pre-processed-wordnet, dtype: object

## Basic and Deep neural network

In [0]:
train_final = df[df.source=="train"]
test_final = df[df.source=="test"]

In [0]:
# build train and test datasets
reviews = train_final['Text'].values
sentiments = train_final['Sentiment'].values

train_reviews = reviews[:7400]
train_sentiments = sentiments[:7400]

test_reviews = reviews[7400:]
test_sentiments = sentiments[7400:]

In [0]:
test_reviews.shape

(7425,)

In [0]:
import contractions
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata


def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()  
    norm_docs.append(doc)
  
  return norm_docs

In [0]:
%%time

norm_train_reviews = pre_process_corpus(train_reviews)
norm_test_reviews = pre_process_corpus(test_reviews)

100%|██████████| 7425/7425 [00:01<00:00, 4489.47it/s]

CPU times: user 3.24 s, sys: 36.2 ms, total: 3.28 s
Wall time: 3.3 s


In [0]:
%%time

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# build BOW features on train reviews
cv = CountVectorizer(binary=False,min_df=5, max_df=1.0,  ngram_range=(1,2))
cv_train_features = cv.fit_transform(norm_train_reviews)


# build TFIDF features on train reviews
tv = TfidfVectorizer(use_idf=True,min_df=5, max_df=1.0,  ngram_range=(1,2),
                     sublinear_tf=True)
tv_train_features = tv.fit_transform(norm_train_reviews)

CPU times: user 3.22 s, sys: 74.9 ms, total: 3.29 s
Wall time: 3.3 s


In [0]:
%%time

# transform test reviews into features
cv_test_features = cv.transform(norm_test_reviews)
tv_test_features = tv.transform(norm_test_reviews)

CPU times: user 1.64 s, sys: 4.94 ms, total: 1.65 s
Wall time: 1.65 s


In [0]:
print('BOW model:> Train features shape:', cv_train_features.shape, ' Test features shape:', cv_test_features.shape)
print('TFIDF model:> Train features shape:', tv_train_features.shape, ' Test features shape:', tv_test_features.shape)

BOW model:> Train features shape: (7400, 19814)  Test features shape: (7425, 19814)
TFIDF model:> Train features shape: (7400, 19814)  Test features shape: (7425, 19814)


### Model Training, Prediction and Performance Evaluation

In [0]:
%%time

# Logistic Regression model on BOW features
from sklearn.linear_model import LogisticRegression

# instantiate model
lr = LogisticRegression(penalty='l2', max_iter=500, C=1, solver='lbfgs', random_state=42)

# train model
lr.fit(cv_train_features, train_sentiments)

# predict on test data
lr_bow_predictions = lr.predict(cv_test_features)

CPU times: user 1.3 s, sys: 870 ms, total: 2.17 s
Wall time: 1.12 s


In [0]:
from sklearn.metrics import confusion_matrix, classification_report

labels = ['negative', 'positive']
print(classification_report(test_sentiments, lr_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, lr_bow_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.79      0.61      0.68      1201
           1       0.93      0.97      0.95      6224

    accuracy                           0.91      7425
   macro avg       0.86      0.79      0.82      7425
weighted avg       0.90      0.91      0.90      7425



,negative,positive
negative,729,472
positive,199,6025


In [0]:
from sklearn.metrics import confusion_matrix, classification_report

labels = ['negative', 'positive']
print(classification_report(test_sentiments, lr_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, lr_bow_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.79      0.61      0.68      1201
           1       0.93      0.97      0.95      6224

    accuracy                           0.91      7425
   macro avg       0.86      0.79      0.82      7425
weighted avg       0.90      0.91      0.90      7425



,negative,positive
negative,729,472
positive,199,6025


In [0]:
%%time

# Random Forest model on BOW features
from sklearn.ensemble import RandomForestClassifier

# instantiate model
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)

# train model
rf.fit(cv_train_features, train_sentiments)

# predict on test data
rf_bow_predictions = rf.predict(cv_test_features)

CPU times: user 8.42 s, sys: 26 ms, total: 8.44 s
Wall time: 4.37 s


In [0]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, rf_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, rf_bow_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.96      0.15      0.26      1201
           1       0.86      1.00      0.92      6224

    accuracy                           0.86      7425
   macro avg       0.91      0.58      0.59      7425
weighted avg       0.88      0.86      0.82      7425



,negative,positive
negative,183,1018
positive,7,6217


In [0]:
%%time

# Random Forest model on TF-IDF features

# train model
rf.fit(tv_train_features, train_sentiments)

# predict on test data
rf_tfidf_predictions = rf.predict(tv_test_features)

CPU times: user 10.3 s, sys: 23.4 ms, total: 10.3 s
Wall time: 5.28 s


In [0]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, rf_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, rf_tfidf_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.97      0.15      0.25      1201
           1       0.86      1.00      0.92      6224

    accuracy                           0.86      7425
   macro avg       0.91      0.57      0.59      7425
weighted avg       0.88      0.86      0.81      7425



,negative,positive
negative,175,1026
positive,6,6218


In [0]:
import gensim
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Dense
from sklearn.preprocessing import LabelEncoder

In [0]:
le = LabelEncoder()
# tokenize train reviews & encode train labels
tokenized_train = [nltk.word_tokenize(text)
                       for text in norm_train_reviews]
y_train = le.fit_transform(train_sentiments)
# tokenize test reviews & encode test labels
tokenized_test = [nltk.word_tokenize(text)
                       for text in norm_test_reviews]
y_test = le.fit_transform(test_sentiments)

In [0]:
# print class label encoding map and encoded labels
print('Sentiment class label map:', dict(zip(le.classes_, le.transform(le.classes_))))
print('Sample test label transformation:\n'+'-'*35,
      '\nActual Labels:', test_sentiments[:3], '\nEncoded Labels:', y_test[:3])


Sentiment class label map: {0: 0, 1: 1}
Sample test label transformation:
----------------------------------- 
Actual Labels: [1 1 0] 
Encoded Labels: [1 1 0]


### Feature Engineering with word embeddings

In [0]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [0]:
%%time
# build word2vec model
w2v_num_features = 300
w2v_model = gensim.models.Word2Vec(tokenized_train, size=w2v_num_features, window=150,
                                   min_count=10, workers=4, iter=5)

2019-09-15 09:15:58,283 : INFO : collecting all words and their counts
2019-09-15 09:15:58,285 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-09-15 09:15:58,394 : INFO : collected 24018 word types from a corpus of 576790 raw words and 7400 sentences
2019-09-15 09:15:58,395 : INFO : Loading a fresh vocabulary
2019-09-15 09:15:58,421 : INFO : effective_min_count=10 retains 3380 unique words (14% of original 24018, drops 20638)
2019-09-15 09:15:58,422 : INFO : effective_min_count=10 leaves 537209 word corpus (93% of original 576790, drops 39581)
2019-09-15 09:15:58,439 : INFO : deleting the raw counts dictionary of 24018 items
2019-09-15 09:15:58,441 : INFO : sample=0.001 downsamples 59 most-common words
2019-09-15 09:15:58,442 : INFO : downsampling leaves estimated 378705 word corpus (70.5% of prior 537209)
2019-09-15 09:15:58,454 : INFO : estimated required memory for 3380 words and 300 dimensions: 9802000 bytes
2019-09-15 09:15:58,455 : INFO : resetting

CPU times: user 28 s, sys: 55.8 ms, total: 28 s
Wall time: 14.4 s


In [0]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [0]:
# generate averaged word vector features from word2vec model
avg_wv_train_features = averaged_word2vec_vectorizer(corpus=tokenized_train, model=w2v_model,
                                                     num_features=w2v_num_features)
avg_wv_test_features = averaged_word2vec_vectorizer(corpus=tokenized_test, model=w2v_model,
                                                    num_features=w2v_num_features)

In [0]:
print('Word2Vec model:> Train features shape:', avg_wv_train_features.shape, ' Test features shape:', avg_wv_test_features.shape)

Word2Vec model:> Train features shape: (7400, 300)  Test features shape: (7425, 300)


In [0]:
def construct_deepnn_architecture(num_input_features):
    dnn_model = Sequential()
    dnn_model.add(Dense(512, input_shape=(num_input_features,)))
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))
    
    dnn_model.add(Dense(256))
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))
    
    dnn_model.add(Dense(256))
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))
    
    dnn_model.add(Dense(1))
    dnn_model.add(Activation('sigmoid'))

    dnn_model.compile(loss='binary_crossentropy', optimizer='adam',                 
                      metrics=['accuracy'])
    return dnn_model

In [0]:
w2v_dnn = construct_deepnn_architecture(num_input_features=w2v_num_features)

In [0]:
w2v_dnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               154112    
_________________________________________________________________
activation_4 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)              

In [0]:
batch_size = 100
w2v_dnn.fit(avg_wv_train_features, y_train, epochs=10, batch_size=batch_size, 
            shuffle=True, validation_split=0.1, verbose=1)

Train on 6660 samples, validate on 740 samples
Epoch 1/10
6660/6660 [==============================] - 1s 122us/sample - loss: 0.3713 - acc: 0.8449 - val_loss: 0.3648 - val_acc: 0.8324
Epoch 2/10
6660/6660 [==============================] - 1s 93us/sample - loss: 0.3268 - acc: 0.8646 - val_loss: 0.3548 - val_acc: 0.8311
Epoch 3/10
6660/6660 [==============================] - 1s 92us/sample - loss: 0.3215 - acc: 0.8671 - val_loss: 0.3537 - val_acc: 0.8392
Epoch 4/10
6660/6660 [==============================] - 1s 91us/sample - loss: 0.3156 - acc: 0.8701 - val_loss: 0.3515 - val_acc: 0.8405
Epoch 5/10
6660/6660 [==============================] - 1s 91us/sample - loss: 0.3138 - acc: 0.8712 - val_loss: 0.3509 - val_acc: 0.8500
Epoch 6/10
6660/6660 [==============================] - 1s 91us/sample - loss: 0.3086 - acc: 0.8727 - val_loss: 0.3466 - val_acc: 0.8419
Epoch 7/10
6660/6660 [==============================] - 1s 93us/sample - loss: 0.3072 - acc: 0.8707 - val_loss: 0.3515 - val_acc: 

In [0]:
y_pred = w2v_dnn.predict_classes(avg_wv_test_features)
predictions = le.inverse_transform(y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, predictions))
pd.DataFrame(confusion_matrix(test_sentiments, predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.63      0.22      0.33      1201
           1       0.87      0.97      0.92      6224

    accuracy                           0.85      7425
   macro avg       0.75      0.60      0.62      7425
weighted avg       0.83      0.85      0.82      7425



,negative,positive
negative,265,936
positive,156,6068


# Using Lexican Model

In [0]:
!pip install textblob
!pip install textsearch
!pip install contractions
!pip install afinn
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

     |████████████████████████████████| 61kB 7.6MB/s 
  Created wheel for afinn: filename=afinn-0.1-cp36-none-any.whl size=53453 sha256=422e4c1230466fbc6fe891284a2656f8aea5db0eaa62d08495f92db46419a9aa
  Stored in directory: /root/.cache/pip/wheels/b5/1c/de/428301f3333ca509dcf20ff358690eb23a1388fbcbbde008b2
Successfully built afinn
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [0]:
import textblob
from sklearn.metrics import confusion_matrix, classification_report
np.set_printoptions(precision=2, linewidth=80)

## Using TextBlob

In [0]:
reviews = np.array(train_df['Text'])
sentiments = np.array(train_df['Sentiment'])

# extract data for model evaluation
test_reviews = reviews[7400:]
test_sentiments = sentiments[7400:]
sample_review_ids = [4626, 3533, 4]


### Prediction

In [0]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    print('Predicted Sentiment polarity:', textblob.TextBlob(review).sentiment.polarity)
    print('-'*60)

REVIEW: I am a firm believer and user of Maple Syrup over Corn Syrup or white sugar.  It is, simply, a healthier sugar - if sugar can be healthy.<br /><br />I'm sure the syrup will meet all of my taste expectations.<br /><br />The syrup was securely packaged and arrived sooner than I expected.  No complaints.
Actual Sentiment: 1
Predicted Sentiment polarity: 0.09999999999999999
------------------------------------------------------------
REVIEW: I really wanted to find a rice cereal that had more nutritional value than just plain white rice and I am so happy I found this product. I started my baby on this at 5 months and he loves it. I mix it with Breast Milk and once he starts veggies and fruit I will add that in as well. This does not constipate him either and I love that it is Organic and enriched with Probiotcs and DHA.<br />If you are looking for a great first food for your baby with quality ingredients this is it and you can't beat the price for organic.
Actual Sentiment: 1
Predi

In [0]:
sentiment_polarity = [textblob.TextBlob(review).sentiment.polarity for review in test_reviews]

In [0]:
predicted_sentiments = [1 if score >= 0.1 else 0 for score in sentiment_polarity]

### Evaluate Model Performance

In [0]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, predicted_sentiments))
pd.DataFrame(confusion_matrix(test_sentiments, predicted_sentiments), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.44      0.65      0.52      1201
           1       0.93      0.84      0.88      6224

    accuracy                           0.81      7425
   macro avg       0.68      0.74      0.70      7425
weighted avg       0.85      0.81      0.82      7425



,negative,positive
negative,780,421
positive,998,5226


## Using Afinn

In [0]:
from afinn import Afinn

afn = Afinn(emoticons=True)

### Prediction

In [0]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    print('Predicted Sentiment polarity:', afn.score(review))
    print('-'*60)

REVIEW: I am a firm believer and user of Maple Syrup over Corn Syrup or white sugar.  It is, simply, a healthier sugar - if sugar can be healthy.<br /><br />I'm sure the syrup will meet all of my taste expectations.<br /><br />The syrup was securely packaged and arrived sooner than I expected.  No complaints.
Actual Sentiment: 1
Predicted Sentiment polarity: -1.0
------------------------------------------------------------
REVIEW: I really wanted to find a rice cereal that had more nutritional value than just plain white rice and I am so happy I found this product. I started my baby on this at 5 months and he loves it. I mix it with Breast Milk and once he starts veggies and fruit I will add that in as well. This does not constipate him either and I love that it is Organic and enriched with Probiotcs and DHA.<br />If you are looking for a great first food for your baby with quality ingredients this is it and you can't beat the price for organic.
Actual Sentiment: 1
Predicted Sentiment 

In [0]:
sentiment_polarity = [afn.score(review) for review in test_reviews]
predicted_sentiments = [1 if score >= 1.0 else 0 for score in sentiment_polarity]

### Evaluation

In [0]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, predicted_sentiments))
pd.DataFrame(confusion_matrix(test_sentiments, predicted_sentiments), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.54      0.40      0.46      1201
           1       0.89      0.93      0.91      6224

    accuracy                           0.85      7425
   macro avg       0.72      0.67      0.69      7425
weighted avg       0.83      0.85      0.84      7425



,negative,positive
negative,484,717
positive,408,5816


### Using VADER

In [0]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.strip()
    doc = contractions.fix(doc)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(test_reviews)
len(norm_corpus)

7425

In [0]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def analyze_sentiment_vader_lexicon(review, 
                                    threshold=0.1,
                                    verbose=False):    
    # analyze the sentiment for review
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    # get aggregate scores and final sentiment
    agg_score = scores['compound']
    final_sentiment = 1 if agg_score >= threshold\
                                   else 0
    if verbose:
        # display detailed sentiment statistics
        positive = str(round(scores['pos'], 2)*100)+'%'
        final = round(agg_score, 2)
        negative = str(round(scores['neg'], 2)*100)+'%'
        neutral = str(round(scores['neu'], 2)*100)+'%'
        sentiment_frame = pd.DataFrame([[final_sentiment, final, positive,
                                        negative, neutral]],
                                        columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'], 
                                                                      ['Predicted Sentiment', 'Polarity Score',
                                                                       'Positive', 'Negative', 'Neutral']], 
                                                              codes=[[0,0,0,0,0],[0,1,2,3,4]]))
        print(sentiment_frame)
    
    return final_sentiment

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


### Prediction

In [0]:
for review, sentiment in zip(norm_corpus[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    pred = analyze_sentiment_vader_lexicon(review, threshold=0.4, verbose=True)    
    print('-'*60)

REVIEW: I am a firm believer and user of Maple Syrup over Corn Syrup or white sugar  It is simply a healthier sugar  if sugar can be healthybr br I am sure the syrup will meet all of my taste expectationsbr br The syrup was securely packaged and arrived sooner than I expected  No complaints
Actual Sentiment: 1
     SENTIMENT STATS:                                         
  Predicted Sentiment Polarity Score Positive Negative Neutral
0                   0          -0.05     9.0%     9.0%   82.0%
------------------------------------------------------------
REVIEW: I really wanted to find a rice cereal that had more nutritional value than just plain white rice and I am so happy I found this product I started my baby on this at 5 months and he loves it I mix it with Breast Milk and once he starts veggies and fruit I will add that in as well This does not constipate him either and I love that it is Organic and enriched with Probiotcs and DHAbr If you are looking for a great first food for 

In [0]:
predicted_sentiments = [analyze_sentiment_vader_lexicon(review, threshold=0.4, verbose=False) for review in test_reviews]

### Evaluation

In [0]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, predicted_sentiments))
pd.DataFrame(confusion_matrix(test_sentiments, predicted_sentiments), index=labels, columns=labels)

              precision    recall  f1-score   support

           0       0.50      0.57      0.54      1201
           1       0.92      0.89      0.90      6224

    accuracy                           0.84      7425
   macro avg       0.71      0.73      0.72      7425
weighted avg       0.85      0.84      0.84      7425



,negative,positive
negative,690,511
positive,684,5540
